# Necessary Libraries & Global Variables

In [19]:
%matplotlib inline
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats as ss
import pandas as pd
import warnings
import random
import logging

import time
import pdb

Zero = 0

# Global Variables
* cnt = 0 # count of all non-zero data points
* dsum = 0 # sum of all non-zero data point values for all features
* d2sum = 0 # sum of all non-zero data point values squared for all features


* cnt0 = 0 # count of all data points
* dsum0 = 0 # sum of all data point values for all features
* d2sum0 = 0 # sum of all data point values squared for all features


* lndsum = 0 # sum of log of all non-zero data point values for all features
* lnd2sum = 0 # sum of log of all non-zero data point values squared for all features
* xmin = -1 # minimum among all of non-zero data point values


In [93]:
def init_default():
    return {'cnt':0, 'dsum':0, 'd2sum':0, 'lndsum':0, 'lnd2sum':0, 'xmin':float("inf"), \
            'cnt0':0, 'dsum0':0, 'd2sum0':0}

def update_default(data, default):
    data = np.asarray(data)
    default['cnt0'] += len(data)
    default['dsum0'] += np.sum(data)
    default['d2sum0'] += np.sum(data**2)
    data = data[data > 0]
    default['cnt'] += len(data)
    default['dsum'] += np.sum(data)
    default['d2sum'] += np.sum(data**2)
    if len(data) > 0:
        default['xmin'] = min(min(data), default['xmin'])
    lndata = np.log(data)
    default['lndsum'] += np.sum(lndata)
    default['lnd2sum'] += np.sum(lndata**2)
    return default
    
def use_default(dist, default):
    loc = -1
    scale = -1
    shape = -1
    if dist == 'norm':
        loc = default['dsum0']/default['cnt0']
        scale = np.sqrt(default['d2sum0']/default['cnt0']-loc**2)        
    elif dist == "lognorm":
        loc = default['lndsum']/default['cnt']
        scale = np.sqrt(default['lnd2sum']/default['cnt']-loc**2)
    elif dist == "powerlaw":
        shape = 1 + default['cnt']/(default['lndsum'] - default['cnt']*np.log(default['xmin']))
    elif dist == "zin_norm":
        loc = default['dsum']/default['cnt']
        scale = np.sqrt(default['d2sum']/default['cnt']-loc**2)
    elif dist == "zin_lognorm":
        loc = default['lndsum']/default['cnt']
        scale = np.sqrt(default['lnd2sum']/default['cnt']-loc**2)
    elif dist == "zin_powerlaw":
        shape = 1 + default['cnt']/(default['lndsum'] - default['cnt']*np.log(default['xmin']))
    return (loc, scale, shape, default['xmin'])

# Maximum Likelihood Estimation
The following methods calculate MLE of different distributions based on the input empirical data

MLE calculator returns at most 4 parameters for the distributions "normal", "log-normal", "power law", "fisk", "binomial", "zero-inflated log-normal", and "zero-inflated power law".
1. locale -> key: loc
2. scale -> key: scale
3. shape -> key: shape

input zero shows the real number that should be considered as zero if the data is normalized into 0.005 to 1 of range of x.

NOTE: WHEN N, COUNT OF DATA TO CALCULATE MLE, AND N', COUNT OF OUT-OF-SAMPLE DATA TO CALCULATE LIKELIHOOD ON ARE NOT EQUAL, VALUE OF ZERO IS NOT THE SAME. DON'T KNOW WHAT WE SHOULD DO IN THIS CASE!!!

In [1]:
def mle(data, dist, zero = 0): 
    data = np.asarray(data) # data is of type np.array
    loc = -1
    scale = -1
    shape = -1
    p0 = -1 # Probability of being zero in zero-inflated distributions
    x_min = -1
    cnt = len(data)
    warningMsg = "Data out of range for " + dist + ". Will remove "
    #logging.warn(warningMsg.format("x<=0"))
    if dist == 'norm': # MLE for Normal
        if len(data) > 1:
            loc = np.mean(data)
            scale = np.std(data)
    
    elif dist == 'lognorm': # MLE for Log-normal
        if len(data) > 1:
            if min(data) <= 0:
                logging.warn(warningMsg+"x <= 0")
                data = data[data > 0]
            if len(data) > 1: # O.W. we will use default in pdmf
                lndata = np.log(data)
                loc = np.mean(lndata)
                scale = np.std(lndata)
            
    
    elif dist == 'powerlaw': # MLE for Power Law
        if len(data) != 0:
            if min(data) <= 0:
                logging.warn(warningMsg+"x <= 0")
                data = data[data > 0]
            # x_min is just set to the smallest non-zero x
            if len(data) != 0: # O.W. we will use default in pdmf
                x_min = min(data)
                shape = 1 + len(data)/(np.sum(np.log(data/x_min)))
    
    elif dist == 'zin_norm': # MLE for Zero-Inflated Normal
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        data = data[data >= zero]
        p0 = (len(data[data == zero])+1)/(len(data)+2) # laplace smoothing to scape zeros
        data = data[data > zero]
        if len(data) != 0: # O.W. we will use default in pdmf
            # normal MLE
            loc = np.mean(data)
            scale = np.std(data)     
        
    elif dist == 'zin_lognorm': # MLE for Zero-Inflated Log-normal
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        data = data[data >= zero]
        p0 = (len(data[data == zero])+1)/(len(data)+2)
        data = data[data > zero]
        if len(data) != 0: # O.W. we will use default in pdmf
            # log-normal MLE
            lndata = np.log(data)
            loc = np.mean(lndata)
            scale = np.std(lndata)
        
    elif dist == 'zin_powerlaw': # MLE for Zero-Inflated Power Law
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        data = data[data >= zero]
        p0 = (len(data[data == zero])+1)/(len(data)+2)
        data = data[data > zero]
        if len(data) != 0: # O.W. we will use default in pdmf
            # power law MLE
            x_min = min(data)
            shape = 1 + len(data)/(np.sum(np.log(data/x_min)))

    elif dist == 'binomial' or dist == 'bernoulli': # MLE for Binomial or Bernoulli
        #if not np.issubdtype(data, np.integer):
        #    logging.warn("Can't calculate MLE. Not integer values for Binomial distribution.")
        #else:
        if min(data) < zero:
            logging.warn(wanringMsg.format("x < " + str(zero)))
            data = data[data >= zero]
        p0 = (len(data[data == zero]) + 1)/(len(data) + 2)
        
    return {'loc':loc, 'scale':scale, 'shape':shape, 'xmin':x_min, 'p0':p0, 'logp0':np.log(p0), 'cnt':cnt}

In [82]:
#nCr
import operator as op
def ncr(nn, rr):
    pdb.set_trace()
    rr = min(rr, nn-rr)
    if rr == 0: return 1
    numer = reduce(op.mul, xrange(nn, nn-rr, -1))
    denom = reduce(op.mul, xrange(1, rr+1))
    return numer//denom

# pdf or pmf 
Results of out-of-sample data based for the theoretical distribution given its parameters.

In [62]:
def pdmf(data, dist, params, default, zero = 0):
    # IMPORTANT: we assume data is sorted in ascending order
    loc = params['loc']
    scale = params['scale']
    shape = params['shape']
    p0 = params['p0']
    x_min = params['xmin']
    cnt = params['cnt']
    
    if loc == -1 and scale == -1 and shape == -1:
        loc, scale, shape, x_min = use_default(dist, default)
    
    ll = None
    warningMsg = "Data out of range for " + dist + ". Will remove "
    
    if dist == 'norm': # pdf for Normal
        pdmf_arr = np.exp(-(data-loc)**2/(2*scale**2))/(scale * np.sqrt(2*np.pi))

    elif dist == 'lognorm': # pdf for Log-normal
        if min(data) <= 0:
            logging.warn(warningMsg+"x <= 0")
            data = data[data > 0]
        lndata = np.log(data)
        pdmf_arr = np.exp(-(lndata-loc)**2/(2*scale**2))/(data*scale*np.sqrt(2*np.pi))
    
    elif dist == 'powerlaw': # pdf for Power Law
        if min(data) <= 0:
            logging.warn(warningMsg+"x <= 0")
            data = data[data > 0]
        # x_min is just set to the smallest non-zero x
        #x_min = min(data)
        pdmf_arr = (shape-1)*(data/x_min)**(-shape)/x_min
    
    elif dist == 'zin_norm': # pdf for Zero-Inflated Normal
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        zero_count = len(data[data == zero])
        data = data[data > zero] # nonzero data
        pdmf_arr = np.exp(-(data-loc)**2/(2*scale**2))/(scale * np.sqrt(2*np.pi))
        pdmf_arr = np.append([p0]*zero_count, (1 - p0)*pdmf_arr) # data was sorted in ascending order

    elif dist == 'zin_lognorm': # pmf for Zero-Inflated Log-normal
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        zero_count = len(data[data == zero])
        data = data[data > zero] # nonzero data
        lndata = np.log(data)
        pdmf_arr = np.exp(-(lndata-loc)**2/(2*scale**2))/(data*scale*np.sqrt(2*np.pi))
        pdmf_arr = np.append([p0]*zero_count, (1 - p0)*pdmf_arr) # data was sorted in ascending order
        
    elif dist == 'zin_powerlaw': # pmf for Zero-Inflated Power Law
        if min(data) < 0:
            logging.warn(warningMsg+"x < 0")
        zero_count = len(data[data == zero])
        data = data[data > zero]
        #x_min = min(data)
        pdmf_arr = (shape-1)*(data/x_min)**(-shape)/x_min
        pdmf_arr = np.append([p0]*zero_count, (1 - p0)*pdmf_arr) # data was sorted in ascending order
        
    elif dist == 'binomial': # pmf for Binomial
        #if not all(item.is_integer() for item in data):
        #if not issubclass(data.dtype, np.integer):
        #    logging.warn("Can't calculate MLE. Not integer values for Binomial distribution.")
        #else:
        if min(data) < 0:
            logging.warn(wanringMsg+"x < 0")
            data = data[data >= zero]
        pdmf_arr = []
        hh = ss.binom(cnt, p0)
        for d in data.tolist():
            #pdmf_arr += [ncr(cnt, d)*p0**(cnt-d)*(1-p0)**d]
            pdmf_arr += hh.pmf(d)
        pdmf_arr = np.asarray(pdmf_arr)
    elif dist == 'bernoulli': # pmf for Bernoulli
        #if not all(item.is_integer() for item in data):
        #    logging.warn("Can't calculate MLE. Not integer values for Bernoulli distribution.")
        #else:
        if min(data) < 0:
            logging.warn(wanringMsg+"x < 0")
            data = data[data >= zero]
        pdmf_arr = []
        for d in data.tolist():
            pdmf_arr += [p0 if d == 0 else (1-p0)]
        pdmf_arr = np.asarray(pdmf_arr)
    return pdmf_arr

def llhood(data, dist, params, zero = 0):
    pdmf_arr = pdmf(data, dist, params, zero)
    ll = np.sum(np.log(pdmf_arr))    
    return ll


# PDF or PMF
## For a single value

In [89]:
def singleval_pdmf(val, dist, params, default, zero = 0):
    # IMPORTANT: we assume data is sorted in ascending order
    loc = params['loc']
    scale = params['scale']
    shape = params['shape']
    p0 = params['p0']
    x_min = params['xmin']
    cnt = params['cnt']
    
    if loc == -1 and scale == -1 and shape == -1:
        loc, scale, shape, x_min = use_default(dist, default)
    
    ll = None
    res = -1
    warningMsg = "Data out of range for " + dist + ". Will return -1 "
    
    if dist == 'norm': # pdf for Normal
        res = np.exp(-(val-loc)**2/(2*scale**2))/(scale * np.sqrt(2*np.pi))

    elif dist == 'lognorm': # pdf for Log-normal
        if val <= 0: logging.warn(warningMsg+"x <= 0")
        else: res = np.exp(-(np.log(val)-loc)**2/(2*scale**2))/(val*scale*np.sqrt(2*np.pi))
    
    elif dist == 'powerlaw': # pdf for Power Law
        if val <= 0: logging.warn(warningMsg+"x <= 0")
        else: res = (shape-1)*(val/x_min)**(-shape)/x_min
    
    elif dist == 'zin_norm': # pdf for Zero-Inflated Normal
        if val < 0: logging.warn(warningMsg+"x < 0")
        else:
            if val == zero: res = p0
            else: res = np.exp(-(val-loc)**2/(2*scale**2))/(scale * np.sqrt(2*np.pi))

    elif dist == 'zin_lognorm': # pmf for Zero-Inflated Log-normal
        if val < 0: logging.warn(warningMsg+"x < 0")
        else:
            if val == zero: res = p0
            else: res = np.exp(-(np.log(val)-loc)**2/(2*scale**2))/(val*scale*np.sqrt(2*np.pi))
        
    elif dist == 'zin_powerlaw': # pmf for Zero-Inflated Power Law
        if val < 0: logging.warn(warningMsg+"x < 0")
        else:
            if val == zero: res = p0
            else: res = (shape-1)*(val/x_min)**(-shape)/x_min
        
    elif dist == 'binomial': # pmf for Binomial
        #if not all(item.is_integer() for item in data):
        #if not issubclass(data.dtype, np.integer):
        #    logging.warn("Can't calculate MLE. Not integer values for Binomial distribution.")
        #else:
        pdb.set_trace()
        if val < 0: logging.warn(wanringMsg+"x < 0")
        else: res = ncr(cnt, val)*p0**(cnt-val)*(1-p0)**val
    elif dist == 'bernoulli': # pmf for Bernoulli
        #if not all(item.is_integer() for item in data):
        #    logging.warn("Can't calculate MLE. Not integer values for Bernoulli distribution.")
        #else:
        if val < 0: logging.warn(wanringMsg+"x < 0")
        else: res = p0 if val == 0 else (1-p0)
    return res

# Find Best Fit Distribution
## Function name: best_fit_distribution
**Args:**
* data -> a series of data values (like group norms in our case)
* bins -> how many bins we want to divide the data into (default = 200)
* ax -> axes subplots (default None)
  * subplot of the data with x and y limits set
  * If given, this function will add histogram of data + all different distribution lines to this subplot

**Returns:**
* Best_Distribution_name -> winner among all distributions
* Best_SSE -> Minimum sum of squared error
* Best_params -> parameters of the best distribution which has at least two parameters of "Location" and "Scale" + any additional parameters based on the distribution type

**What it does?**
* Creates a histogram of the data divided to <BINS> bins
* Sets x as the center of each bin
* Sets y as the frequency value of each bin (normed = True)
* Finds the best fit distribution among previously defined distributions based on the SSE of the returned value and the actual value
  * Uses scipy.stats distribution methods **fit** and **pdf**
  * scipy.stats current Distributions are **norm**, **expon**, and **lognorm** for now
* If ax is set, it adds histogram and all the distributions lines with their MLE parameters to that

In [187]:
# Create models from data
def best_fit_distribution(train, test, ax=None, zero=0):
    """Model data by finding best fit distribution to data"""
    DISTRIBUTIONS = [
        'norm', 'lognorm', 'powerlaw', 'zin_lognorm', 'zin_powerlaw'
    ]
    # Best holders
    best_distribution = ss.norm
    best_params = (0.0, 1.0)
    best_ll = -1 * np.inf

    dist_ll = {}
    
    # Estimate distribution parameters from data
    for distribution in DISTRIBUTIONS:
        # Try to fit the distribution
        try:
            params = mle(train, distribution, zero)
            ll = llhood(test, distribution, params, zero)

            dist_ll[distribution] = [round(ll, 2), params]
    
            # identify if this distribution is better
            if best_ll < ll:
                best_distribution = distribution
                best_params = params
                best_ll = ll
        except Exception as detail:
            print('Runtime Error: ' + str(detail))
            pass
                
        if ax:
             # Calculate fitted PDF and error with fit in distribution
            y = pdmf(test, distribution, params, zero)
            pd.Series(y, test). \
            plot(ax=ax, label=distribution + str(round(ll, 2)))

    return (best_distribution, best_ll, best_params, dist_ll)

Test 

Will be commented for the sake of not being imported in other notebooks.
Whenever you want to try the codes in this notebook, you can comment it out

#data = ss.lognorm.rvs(1,size=100)
#data = np.sort(data)
#data = np.append(data , [0] * 5)
#data = data + 0.00000001
val_idx = plt.hist(data, bins=105, normed=1, color='grey')
idx = val_idx[1]
params = mle(data + 0.1, 'lognorm', 0)
#params1 = mle(data+0.000001, 'lognorm', 0)
params2 = ss.lognorm.fit(data)
print params
print params2
#print params1
ll = llhood(data + 0.1, 'lognorm', params, 0)
idx = (idx[1:] + idx[:-1])/2
pdmf_arr = pdmf(idx + 0.1, 'lognorm', params, 0)
pdf = ss.lognorm.pdf(idx, loc=params2[-2], scale=params2[-1], *params2[:-2])
#pdb.set_trace()
plt.plot(idx, pdmf_arr, color='green')
#plt.plot(idx, pdf, color='red')
plt.title('ll: ' + str(ll))